# Prediction metrics

In [1]:
def LHT(op, l, r):
    if op in Locks[l]:
        return service_time[op] + lock_cost[l][op][PLACE][r]
    else:
        return 0

    
def op_serialization(l, r):
    res = 0
    for op in Ops:
        if modes[MODE][op] == 'X':
            for r_1 in Reps:
                if r_1 != r:
                    res += LHT(op, l, r_1) * freq[WL][op][r_1]
#     print('os', res)
    return res


def rep_serialization(l, r):
    res = 0
    for op in Ops:
        res += LHT(op, l, r) * freq[WL][op][r]
#     print('rp', res)
    return res


def op_parallelism(l, r):
    res = 0
    s = 0
    for r_1 in Reps:
        if r != r_1:
            for op in Ops:
                if modes[MODE][op] == 'S':
                    s += LHT(op, l, r_1) * freq[WL][op][r_1]
        res = max(res, s)
#     print('op', res)
    return res


def nonparallelism(l):
    res = 0
    for r in Reps:
        np = abs(op_serialization(l, r) + rep_serialization(l, r) - op_parallelism(l,r))
        res = max(res, np)
#     print('np', res)
    return res


def coordination_parallelism():
    res = 0
    for l in Locks:
        np = nonparallelism(l)
        res = max(res, np)
    return res


def max_replica_exec_time(WL, l):
    res = 0
    for r in Reps:
        repexec = rep_serialization(l, r) + op_serialization(l, r)
        res = max(res, repexec)
    return res


In [2]:
Locks = {'a-b':{'insert', 'read'}}
Reps = ['houston', 'paris', 'singapore']
Ops = ['insert', 'read']
modes = {1:{'insert':'X', 'read':'X'}, 2:{'insert':'X','read':'S'}, 3:{'insert':'S','read':'X'}}
service_time = {'insert':0, 'read':0}
lock_cost = {'a-b': {'insert': {'houston':{'houston':3.2954730850000002, 'paris':223.78096148900002, 'singapore':503.93539986400003}, 
                                 'paris':{'houston':223.99058557599997, 'paris':3.2398566490000005, 'singapore':303.88539813499995}, 
                                 'singapore':{'houston':503.86683250899995, 'paris':303.965657338, 'singapore':3.108215382}},
                     'read': {'houston':{'houston':3.0035555910000005, 'paris':223.53921476500003, 'singapore':503.69265980899996}, 
                                 'paris':{'houston':223.67417935400002, 'paris':3.0147095129999997, 'singapore':303.628741349}, 
                                 'singapore':{'houston':503.62028737099996, 'paris':303.671423894, 'singapore':2.9168805460000002}}
            }}
freq = {'workloadeqeq':{'insert':{'houston':167, 'paris':167, 'singapore':166}, 'read':{'houston':167, 'paris':167, 'singapore':166}},
       'workloadeqhot':{'insert':{'houston':0, 'paris':500, 'singapore':0}, 'read':{'houston':0, 'paris':500, 'singapore':0}},
        'workloadeqclust':{'insert':{'houston':250, 'paris':250, 'singapore':0}, 'read':{'houston':250, 'paris':250, 'singapore':0}},
        'workloadhoteq':{'insert':{'houston':333, 'paris':334, 'singapore':333}, 'read':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadhothot':{'insert':{'houston':0, 'paris':1000, 'singapore':0}, 'read':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadhotclust':{'insert':{'houston':500, 'paris':500, 'singapore':0}, 'read':{'houston':0, 'paris':0, 'singapore':0}},
        'workloadF':{'insert':{'houston':0, 'paris':500, 'singapore':0}, 'read':{'houston':167, 'paris':167, 'singapore':166}},
        'workloadG':{'insert':{'houston':0, 'paris':500, 'singapore':0}, 'read':{'houston':250, 'paris':0, 'singapore':250}}
       }

In [18]:
predictions = {}
for WL in freq:
    predictions[WL] = {}
    for GRAN in [1]:
        for MODE in [1,2,3]:
            for i, PLACE in enumerate(Reps):
                predictions[WL][str(GRAN) + '-' + str(MODE) + '-' + str(i+1)] = coordination_parallelism()
#             print(str(MODE) + '-' + str(i+1), '\t', coordination_parallelism())
#             print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', coordination_parallelism())
#             print(coordination_parallelism())
print(predictions)

max_rep_times={}
for WL in freq:
    max_rep_times[WL] = {}
    for l in Locks:
        for GRAN in [1]:
            for MODE in [1,2,3]:
                for i, PLACE in enumerate(Reps):
                    max_rep_times[WL][str(GRAN) + '-' + str(MODE) + '-' + str(i+1)] = max_replica_exec_time(WL, l)
#                     print(max_replica_exec_time(WL, l))

print(max_rep_times)

{'workloadeqeq': {'1-1-1': 243020.66512902803, '1-1-2': 176651.875446708, '1-1-3': 270725.90750975197, '1-2-1': 167355.379830124, '1-2-2': 100937.78656513, '1-2-3': 168331.24758788396, '1-3-1': 167177.135981312, '1-3-2': 100756.907743558, '1-3-3': 168169.450452036}, 'workloadeqhot': {'1-1-1': 223660.08812700002, '1-1-2': 3127.283081, '1-1-3': 303818.540616, '1-2-1': 223660.08812700002, '1-2-2': 3127.283081, '1-2-3': 303818.540616, '1-3-1': 223660.08812700002, '1-3-2': 3127.283081, '1-3-3': 303818.540616}, 'workloadeqclust': {'1-1-1': 113404.80123250002, '1-1-2': 113479.832773, '1-1-3': 403781.050278, '1-2-1': 111903.02343700001, '1-2-2': 111972.47801650001, '1-2-3': 251945.338331, '1-3-1': 111757.06469000001, '1-3-2': 111859.90444849999, '1-3-3': 251798.22160899997}, 'workloadhoteq': {'1-1-1': 243650.721829343, '1-1-2': 176864.81469652895, '1-1-3': 270347.220498595, '1-2-1': 243650.721829343, '1-2-2': 176864.81469652895, '1-2-3': 270347.220498595, '1-3-1': 241455.93675473303, '1-3-2': 

In [3]:
def total_replica_exec_time(WL, l):
    res = 0
    for r in Reps:
        repexec = rep_serialization(l, r) #+ op_serialization(l, r)
        res = res + repexec
    return res

In [4]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in Reps:
            for l in Locks:
                print(total_replica_exec_time(WL, l))

243020.665129028
176651.875446708
270725.90750975197
243020.665129028
176651.875446708
270725.90750975197
243020.665129028
176651.875446708
270725.90750975197
223660.08812700002
3127.283081
303818.540616
223660.08812700002
3127.283081
303818.540616
223660.08812700002
3127.283081
303818.540616
113404.80123250002
113479.832773
403781.05027799995
113404.80123250002
113479.832773
403781.05027799995
113404.80123250002
113479.832773
403781.05027799995
243650.721829343
176864.81469652895
270347.220498595
243650.721829343
176864.81469652895
270347.220498595
243650.721829343
176864.81469652895
270347.220498595
223780.961489
3239.8566490000003
303965.657338
223780.961489
3239.8566490000003
303965.657338
223780.961489
3239.8566490000003
303965.657338
113538.217287
113615.22111249999
403916.2449235
113538.217287
113615.22111249999
403916.2449235
113538.217287
113615.22111249999
403916.2449235
233336.104922246
89879.343829223
287284.746620891
233336.104922246
89879.343829223
287284.746620891
233336

In [5]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in Reps:
            for l in Locks:
                print(max_replica_exec_time(WL, l))

243020.66512902803
176651.875446708
270725.90750975197
205188.022479576
138794.831005919
219528.57754881796
205098.90055517
138704.391595133
219447.67898089398
223660.08812700002
3127.283081
303818.540616
223660.08812700002
3127.283081
303818.540616
223660.08812700002
3127.283081
303818.540616
113404.80123250002
113479.832773
403781.050278
112653.91233475001
112726.15539475
327863.1943045
112580.93296125002
112669.86861075
327789.6359435
243650.721829343
176864.81469652895
270347.220498595
243650.721829343
176864.81469652895
270347.220498595
167810.48815471202
101193.83757895498
167787.65522549697
223780.961489
3239.8566490000003
303965.657338
223780.961489
3239.8566490000003
303965.657338
223780.961489
3239.8566490000003
303965.657338
113538.217287
113615.22111249999
403916.2449235
113538.217287
113615.22111249999
403916.2449235
111890.4807445
111995.29278799999
251933.41625449999
233336.104922246
89879.343829223
287284.74662089103
195503.462272794
52022.299388433996
236087.416659957


In [7]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in Reps:
            for l in Locks:
                for r in Reps:
#                     print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', l, '\t', r[:4], '\t', op_serialization(l, r))
                    print(op_serialization(l, r))

241968.727340136
168318.19569461
75754.40722331
101891.85970339799
175607.362897654
75804.52829236399
102475.558489792
169250.514944008
269725.741585704
121024.696946087
84203.620382619
37921.764573858
50986.03215079299
87851.40388160199
37947.483851574994
51278.228528858
84661.72478241498
134908.025804449
120944.030394049
84114.575311991
37832.642649452
50905.827552605
87755.959016052
37857.044440789
51197.329960934
84588.79016159299
134817.71578125498
223660.08812700002
0.0
223660.08812700002
3127.283081
0.0
3127.283081
303818.540616
0.0
303818.540616
111890.4807445
0.0
111890.4807445
1619.9283245000001
0.0
1619.9283245000001
151982.828669
0.0
151982.828669
111769.6073825
0.0
111769.6073825
1507.3547565
0.0
1507.3547565
151835.711947
0.0
151835.711947
111830.04406350001
1574.757169
113404.8012325
1563.6415405
111916.1912325
113479.832773
151909.270308
251871.77996999997
403781.050278
55945.24037225
823.86827125
56769.1086435
809.9641622500001
55997.646393999996
56807.610556249994
759

In [8]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in Reps:
            for l in Locks:
                for r in Reps:
#                     print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', l, '\t', r[:4], '\t', rep_serialization(l, r))
                    print(rep_serialization(l, r))
    

1051.937788892
74702.469434418
167266.257905718
74760.01574331
1044.512549054
100847.347154344
168250.34901995998
101475.392565744
1000.1659240480001
1051.937788892
74702.469434418
167266.257905718
74760.01574331
1044.512549054
100847.347154344
168250.34901995998
101475.392565744
1000.1659240480001
1051.937788892
74702.469434418
167266.257905718
74760.01574331
1044.512549054
100847.347154344
168250.34901995998
101475.392565744
1000.1659240480001
0.0
223660.08812700002
0.0
0.0
3127.283081
0.0
0.0
303818.540616
0.0
0.0
223660.08812700002
0.0
0.0
3127.283081
0.0
0.0
303818.540616
0.0
0.0
223660.08812700002
0.0
0.0
3127.283081
0.0
0.0
303818.540616
0.0
1574.757169
111830.04406350001
0.0
111916.1912325
1563.6415405
0.0
251871.77996999997
151909.270308
0.0
1574.757169
111830.04406350001
0.0
111916.1912325
1563.6415405
0.0
251871.77996999997
151909.270308
0.0
1574.757169
111830.04406350001
0.0
111916.1912325
1563.6415405
0.0
251871.77996999997
151909.270308
0.0
1097.3925373050001
74742.841137

In [9]:
for WL in freq:
    for MODE in [1,2,3]:
        for PLACE in Reps:
            for l in Locks:
                for r in Reps:
#                     print(WL[8:]+' '*(16-len(WL)), '\t', MODE, '\t', PLACE, '\t', l, '\t', r[:4], '\t', op_parallelism(l, r))
                    print(op_parallelism(l, r))
                    

0
0
0
0
0
0
0
0
0
120944.030394049
84114.575311991
37832.642649452
50905.827552605
87755.959016052
37857.044440789
51197.329960934
84588.79016159299
134817.71578125498
121024.696946087
84203.620382619
37921.764573858
50986.03215079299
87851.40388160199
37947.483851574994
51278.228528858
84661.72478241498
134908.025804449
0
0
0
0
0
0
0
0
0
111769.6073825
0
111769.6073825
1507.3547565
0
1507.3547565
151835.711947
0
151835.711947
111890.4807445
0
111890.4807445
1619.9283245000001
0
1619.9283245000001
151982.828669
0
151982.828669
0
0
0
0
0
0
0
0
0
55884.80369125
750.8888977500001
56635.692589000006
753.67737825
55918.544838500005
56672.22221675001
75917.8559735
125905.07184275
201822.92781625
55945.24037225
823.86827125
56769.1086435
809.9641622500001
55997.646393999996
56807.610556249994
75991.4143345
125966.70812724999
201958.12246175
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
242553.32929203802
168907.880692017
75840.233674631
102275.94969972098
175782.70257576296
75670.97711757399
102559.565